## Variable Initialization and Imports

In [ ]:
import pandas as pd
from sklearn import linear_model
import csv
import matplotlib.pyplot as plt
from eyeInfo import EyeInfo

output = './csv/output/cinco.csv'
train_imput = './csv/cinco_fixed_train_data.csv'
predict_input = './csv/cinco_predict_train_data.csv'

fieldnames = ['screen_x', 'screen_y']
csv_file = output

calib_points = [
    { "x": 200, "y": 200 },
    { "x": 1672, "y": 200 },
    { "x": 936, "y": 472 },
    { "x": 200, "y": 744 },
    { "x": 1672, "y": 744 }
]

def train_to_validate_calib():
    dataset_train_path = train_imput
    dataset_predict_path = predict_input

    data = pd.read_csv(dataset_train_path)

    X = data[['left_iris_x', 'left_iris_y', 'right_iris_x', 'right_iris_y']]
    y = data[['point_x', 'point_y']]

    model = linear_model.LinearRegression()
    model.fit(X, y)

    dados_teste = pd.read_csv(dataset_predict_path)

    previsoes = model.predict(dados_teste)

    print("Previsões de point_x e point_y:")
    print(previsoes)
    return previsoes.tolist()

## Reading CSVs

In [ ]:
teste = train_to_validate_calib()

with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()

    for row in teste:
        writer.writerow({'screen_x': row[0], 'screen_y': row[1]})

## Instancing class and initializing calib points 

In [ ]:
eye = EyeInfo(calib_points=calib_points, dataset=output)
eye.get_screen_resolution()
eye.get_calib_points()
eye.get_points()

calib_points_x = []
calib_points_y = []
for obj in eye.calib_points:
    calib_points_x.append(obj['x'])
    calib_points_y.append(obj['y'])

## Initializing left eye, right eye and prediction arrays to plot

In [ ]:
right = eye.right_eye
# print(f'right: {right}')

right_eye_x = []
right_eye_y = []


for obj in right:
    right_eye_x.append(obj['x'])
    right_eye_y.append(obj['y'])

left = eye.left_eye
# print(f'left: {left}')
left_eye_x = []
left_eye_y = []


for obj in left:
    left_eye_x.append(obj['x'])
    left_eye_y.append(obj['y'])

prediction = eye.prediction
# print(f'prediction: {prediction}')
prediction_x = []
prediction_y = []

for obj in prediction:
    prediction_x.append(obj['x'])
    prediction_y.append(obj['y'])

## Plotting data

In [ ]:
# plt.scatter(right_eye_x, right_eye_y, marker='o', color='blue', label='right eye')
# plt.scatter(left_eye_x, left_eye_y, marker='o', color='red', label='left eye')
plt.scatter(prediction_x, prediction_y, marker='o', color='green', label='predicted value')
plt.scatter(calib_points_x, calib_points_y, marker="*", color='black', label='calib points')

fig = plt.gcf()
fig.set_size_inches(eye.k_screen_width / 100, eye.k_screen_height / 100)

ax = plt.gca()
ax.set_xlim(0, eye.k_screen_width)
ax.set_ylim(0, eye.k_screen_height)

plt.grid(True)

plt.show()